In [23]:
from pathlib import Path
import numpy as np
import pandas as pd

data_path = Path('/Users/vinushan/Documents/Y4S2/RP module/contextawareforecastingsys/data/fnb_dataset .xlsx')
df = pd.read_excel(data_path)
df.head()

,SYSTEM DATE,BILL NO,PAYMENT MODE,PAYMENT DATE,ORDER TYPE,ORDER TYPE NAME,ORDER NO,TABLE NAME,DISCOUNT,DISCOUNT REMARKS,...,FOOD ID,FOOD NAME,QTY,GROSS PRICE,TOTAL DISCOUNT PRICE,SERVICE CHARGE PRICE,PACKAGE CHARGE PRICE,TOTAL PRICE,FOOD COST,PAYMENT DATE TIME
0,2025-12-21,3280,CITY LEDGER,2025-12-21,DELIVERY,NaN,7280,TBL 4,NaN,NaN,...,121.0,Deluxe Roll,1,340.33,0.00,0,0,413.15,284.47,2025-12-21 17:35:00
1,2025-05-27,1571,CASH,2025-05-27,DELIVERY,NaN,5571,TBL 7,NaN,NaN,...,103.0,Spicy Drink,1,432.67,0.00,0,0,384.21,0.00,2025-05-27 16:17:00
2,2025-01-28,5410,CARD,2025-01-28,DELIVERY,NaN,9410,TBL 3,NaN,NaN,...,126.0,Herb Burger,3,102.82,0.00,0,0,380.50,95.09,2025-01-28 11:11:00
3,2025-12-20,1243,COMPLIMENTORY,2025-12-20,DELIVERY,NaN,5243,TBL 3,0.25,Awurudu Deal,...,22.0,Deluxe Roll,3,3138.06,848.67,0,0,7716.44,0.00,2025-12-20 16:12:00
4,2025-12-18,1762,CITY LEDGER,2025-12-18,TABLE,NaN,5762,NaN,Flat,pizza deal19',...,99.0,"Deluxe Pizza 22""",1,1880.87,262.67,0,0,1294.17,0.00,2025-12-18 22:18:00


In [20]:
df.dtypes.to_frame(name="dtype")

,dtype
SYSTEM DATE,datetime64[ns]
BILL NO,int64
PAYMENT MODE,object
PAYMENT DATE,datetime64[ns]
ORDER TYPE,object
ORDER TYPE NAME,object
ORDER NO,int64
TABLE NAME,object
DISCOUNT,object
DISCOUNT REMARKS,object


In [12]:
cols_to_drop = [
    "SERVICE CHARGE PRICE",
    "PACKAGE CHARGE PRICE",
    "DISCOUNT REMARKS",
    "GUEST NAME",
    "TABLE NAME",
    "ORDER TYPE NAME",
    "BILL NO",
    "ORDER NO",
]


def describe_col(col_name: str) -> dict:
    series = df[col_name]
    missing_pct = series.isna().mean() * 100
    nunique = series.nunique(dropna=False)
    sample_vals = series.dropna().unique()[:3]
    sample_str = ", ".join(map(str, sample_vals)) if sample_vals.size else ""
    all_zero = bool((series.fillna(0) == 0).all()) if series.dtype != "O" else False
    return {
        "column": col_name,
        "missing_pct": round(missing_pct, 1),
        "nunique": int(nunique),
        "sample_values": sample_str,
        "all_zero": all_zero
    }

evidence_df = pd.DataFrame(describe_col(col) for col in cols_to_drop)
evidence_df

,column,missing_pct,nunique,sample_values,all_zero
0,SERVICE CHARGE PRICE,0.0,1,0,True
1,PACKAGE CHARGE PRICE,0.0,1,0,True
2,DISCOUNT REMARKS,83.6,176,"Awurudu Deal, pizza deal19', com bank",False
3,GUEST NAME,0.0,5657,"Guest2281, Guest572, Guest4411",False
4,TABLE NAME,62.4,13,"TBL 4, TBL 7, TBL 3",False
5,ORDER TYPE NAME,78.2,3,"Uber Eats, Pick Me",False
6,BILL NO,0.0,5657,"3280, 1571, 5410",False
7,ORDER NO,0.0,5657,"7280, 5571, 9410",False


In [30]:
clean_df = df.drop(columns=cols_to_drop).copy()

rename_map = {
    "SYSTEM DATE": "order_date",
    "PAYMENT MODE": "payment_mode",
    "PAYMENT DATE": "payment_date",
    "ORDER TYPE": "order_type",
    "DISCOUNT": "discount_code",
    "ORDER START TIME": "order_start_raw",
    "FOOD ID": "food_id",
    "FOOD NAME": "food_name",
    "QTY": "quantity",
    "GROSS PRICE": "gross_price",
    "TOTAL DISCOUNT PRICE": "total_discount_price",
    "TOTAL PRICE": "total_price",
    "FOOD COST": "food_cost",
    "PAYMENT DATE TIME": "payment_timestamp",
}
clean_df = clean_df.rename(columns=rename_map)

# Ensure temporal columns are datetime
for col in ["order_date", "payment_date", "payment_timestamp"]:
    clean_df[col] = pd.to_datetime(clean_df[col], errors="coerce")

# Numeric coercions
numeric_cols = ["gross_price", "total_discount_price", "total_price", "food_cost"]
clean_df[numeric_cols] = clean_df[numeric_cols].apply(pd.to_numeric, errors="coerce")
clean_df["quantity"] = clean_df["quantity"].astype("Int64")
clean_df["food_id"] = clean_df["food_id"].round().astype("Int64")

# Clean categorical/string fields
for col in ["payment_mode", "order_type", "food_name"]:
    clean_df[col] = (
        clean_df[col]
        .astype(str)
        .str.strip()
        .replace({"": np.nan, "nan": np.nan, "None": np.nan, "NA": np.nan})
    )
    clean_df[col] = clean_df[col].fillna("Unknown")
clean_df["payment_mode"] = clean_df["payment_mode"].str.upper()
clean_df["order_type"] = clean_df["order_type"].str.upper()

clean_df["discount_code"] = (
    clean_df["discount_code"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"": "none", "nan": "none"})
)

# Discount features
percent_discount = pd.to_numeric(clean_df["discount_code"], errors="coerce")
ratio_discount = (
    clean_df["total_discount_price"]
    / clean_df["gross_price"].replace({0: np.nan})
)
clean_df["discount_rate"] = percent_discount.fillna(ratio_discount).round(4)
clean_df["discount_applied"] = clean_df["total_discount_price"].gt(0)
clean_df.loc[clean_df["discount_code"] == "none", "discount_rate"] = clean_df.loc[
    clean_df["discount_code"] == "none",
    "discount_rate",
].fillna(0)
clean_df["discount_rate"] = clean_df["discount_rate"].clip(lower=0).fillna(0)

# Parse order start timestamps
raw_time = clean_df["order_start_raw"].astype(str).str.strip()
missing_time_mask = raw_time.str.lower().isin(["", "nan", "nat", "none"])
order_start_ts = pd.to_datetime(raw_time, errors="coerce")
clean_df["order_start_timestamp"] = order_start_ts
clean_df["order_start_time"] = order_start_ts.dt.time
clean_df.loc[missing_time_mask, "order_start_timestamp"] = pd.NaT
clean_df.loc[missing_time_mask, "order_start_time"] = None

# Additional derived metrics
clean_df["order_weekday"] = clean_df["order_date"].dt.day_name()
clean_df["order_month"] = clean_df["order_date"].dt.to_period("M").astype(str)
clean_df["unit_price"] = (
    clean_df["total_price"]
    / clean_df["quantity"].replace({0: np.nan})
).round(2)
clean_df["gross_margin"] = (
    clean_df["total_price"] - clean_df["food_cost"]
).round(2)

clean_df = (
    clean_df
    .drop(columns=["order_start_raw"])
    .drop_duplicates()
    .sort_values(["order_date", "order_start_timestamp", "food_id"], na_position="last")
    .reset_index(drop=True)
)

column_order = [
    "order_date",
    "order_start_time",
    "order_start_timestamp",
    "payment_date",
    "payment_timestamp",
    "payment_mode",
    "order_type",
    "food_id",
    "food_name",
    "quantity",
    "gross_price",
    "total_discount_price",
    "discount_code",
    "discount_applied",
    "discount_rate",
    "total_price",
    "food_cost",
    "unit_price",
    "gross_margin",
    "order_weekday",
    "order_month",
]
clean_df = clean_df[column_order]
clean_df.head()

/var/folders/5l/dp157sp95rb3x197k0fkfp9c0000gn/T/ipykernel_26383/359687625.py:68: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  order_start_ts = pd.to_datetime(raw_time, errors="coerce")


,order_date,order_start_time,order_start_timestamp,payment_date,payment_timestamp,payment_mode,order_type,food_id,food_name,quantity,...,total_discount_price,discount_code,discount_applied,discount_rate,total_price,food_cost,unit_price,gross_margin,order_weekday,order_month
0,2025-01-01,10:20:00,2025-01-01 10:20:00,2025-01-01,2025-01-01 12:27:00,CARD,DELIVERY,109,"Herb Dessert 19""",1,...,0.00,none,False,0.0000,359.96,286.01,359.96,73.95,Wednesday,2025-01
1,2025-01-01,10:55:00,2025-01-01 10:55:00,2025-01-01,2025-01-01 12:37:00,CARD,TAKEOUT,103,"Deluxe Pizza 19""",4,...,64.81,0.2,True,0.2000,1209.33,0.00,302.33,1209.33,Wednesday,2025-01
2,2025-01-01,11:09:00,2025-01-01 11:09:00,2025-01-01,2025-01-01 11:44:00,CARD,DELIVERY,63,Classic Chicken,2,...,0.00,none,False,0.0000,6661.29,0.00,3330.64,6661.29,Wednesday,2025-01
3,2025-01-01,11:20:00,2025-01-01 11:20:00,2025-01-01,2025-01-01 13:16:00,CITY LEDGER,TAKEOUT,3,"Sweet Chicken 19""",2,...,1016.21,flat,True,0.0959,16737.15,0.00,8368.58,16737.15,Wednesday,2025-01
4,2025-01-01,11:52:00,2025-01-01 11:52:00,2025-01-01,2025-01-01 13:57:00,CITY LEDGER,TAKEOUT,89,"Classic Pasta 19""",5,...,0.00,none,False,0.0000,17689.57,0.00,3537.91,17689.57,Wednesday,2025-01


In [31]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5657 entries, 0 to 5656
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   order_date             5657 non-null   datetime64[ns]
 1   order_start_time       5657 non-null   object        
 2   order_start_timestamp  5657 non-null   datetime64[ns]
 3   payment_date           5657 non-null   datetime64[ns]
 4   payment_timestamp      5657 non-null   datetime64[ns]
 5   payment_mode           5657 non-null   object        
 6   order_type             5657 non-null   object        
 7   food_id                5655 non-null   Int64         
 8   food_name              5657 non-null   object        
 9   quantity               5657 non-null   Int64         
 10  gross_price            5657 non-null   float64       
 11  total_discount_price   5657 non-null   float64       
 12  discount_code          5657 non-null   object        
 13  dis

In [32]:
missing_pct = clean_df.isna().mean().mul(100).round(2)
missing_pct[missing_pct > 0].sort_values(ascending=False)

food_id    0.04
dtype: float64

In [33]:
output_path = data_path.with_name('fnb_dataset_cleaned.csv')
clean_df.to_csv(output_path, index=False)
output_path

PosixPath('/Users/vinushan/Documents/Y4S2/RP module/contextawareforecastingsys/data/fnb_dataset_cleaned.csv')